In [1]:
!pip install dash==0.21.0 \
                  dash-core-components==0.21.1\
                  dash-html-components==0.9.0\
                  dash-renderer==0.12.1\
                  dash-table-experiments==0.6.0

distutils: /usr/local/lib/python3.6/dist-packages
sysconfig: /usr/lib/python3.6/site-packages
distutils: /usr/local/lib/python3.6/dist-packages
sysconfig: /usr/lib/python3.6/site-packages
distutils: /usr/local/include/python3.6/UNKNOWN
sysconfig: /usr/include/python3.6m/UNKNOWN
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 1.9 MB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 75.6 MB/s eta 0:00:01
     |████████████████████████████████| 160 kB 119.2 MB/s eta 0:00:01
     |████████████████████████████████| 738 kB 121.5 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 100.7 MB/s eta 0:00:01
     |████████████████████████████████| 27.7 MB 49.7 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 91.1 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 131.3 MB/s eta 0:00:01
     |██████████████████████████

In [2]:
import base64
import os
from urllib.parse import quote as urlquote

from flask import Flask, send_from_directory
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

ModuleNotFoundError: No module named 'dash'

In [ ]:
UPLOAD_DIRECTORY = "./project/app_uploaded_files"

if not os.path.exists(UPLOAD_DIRECTORY):
    os.makedirs(UPLOAD_DIRECTORY)

In [ ]:
ls {UPLOAD_DIRECTORY}

In [ ]:
#server = Flask(__name__)
#app = dash.Dash(server=server)

portID = '8882'
runID = os.environ['DOMINO_RUN_ID']
user = os.environ['DOMINO_PROJECT_OWNER']
project = os.environ['DOMINO_PROJECT_NAME']

app_path = '<Domino url>'+ user + '/' + project +'/notebookSession/' + runID + '/proxy/' + portID
assets_path = app_path + '/assets/'
print(app_path)

#set the external path to the assets folder
app = dash.Dash(assets_external_path = assets_path)
#app = dash.Dash()

app.scripts.config.serve_locally = True
app.css.config.serve_locally = True    
app = dash.Dash()
 
app.scripts.config.serve_locally = True
# app.css.config.serve_locally = True
 
################################################################
# Configure path for dependencies. This is required for Domino.
#Learn more about Dash on Domino https://blog.dominodatalab.com/building-domino-web-app-dash/
app.config.routes_pathname_prefix=''+portID+'/'
app.config.requests_pathname_prefix=''+portID+'/'
#################################################################


In [ ]:
def download(path):
    """Serve a file from the upload directory."""
    return send_from_directory(UPLOAD_DIRECTORY, path, as_attachment=True)


app.layout = html.Div(
    [
        html.H1("File Browser"),
        html.H2("Upload"),
        dcc.Upload(
            id="upload-data",
            children=html.Div(
                ["Drag and drop or click to select a file to upload."]
            ),
            style={
                "width": "100%",
                "height": "60px",
                "lineHeight": "60px",
                "borderWidth": "1px",
                "borderStyle": "dashed",
                "borderRadius": "5px",
                "textAlign": "center",
                "margin": "10px",
            },
            multiple=True,
        ),
        html.H2("File List"),
        html.Ul(id="file-list"),
    ],
    style={"max-width": "500px"},
)


def save_file(name, content):
    """Decode and store a file uploaded with Plotly Dash."""
    data = content.encode("utf8").split(b";base64,")[1]
    with open(os.path.join(UPLOAD_DIRECTORY, name), "wb") as fp:
        fp.write(base64.decodebytes(data))


def uploaded_files():
    """List the files in the upload directory."""
    files = []
    for filename in os.listdir(UPLOAD_DIRECTORY):
        path = os.path.join(UPLOAD_DIRECTORY, filename)
        if os.path.isfile(path):
            files.append(filename)
    return files


def file_download_link(filename):
    """Create a Plotly Dash 'A' element that downloads a file from the app."""
    location = "./" #download/{}".format(urlquote(filename))
    return html.A(filename, href=location)


In [ ]:
@app.callback(
    Output("file-list", "children"),
    [Input("upload-data", "filename"), Input("upload-data", "contents")],
)

def update_output(uploaded_filenames, uploaded_file_contents):
    """Save uploaded files and regenerate the file list."""

    if uploaded_filenames is not None and uploaded_file_contents is not None:
        for name, data in zip(uploaded_filenames, uploaded_file_contents):
            save_file(name, data)

    files = uploaded_files()
    
    if len(files) == 0:
        return [html.Li("No files yet!")]
    else:
        return [html.Li(file_download_link(filename)) for filename in files]

In [ ]:
if __name__ == '__main__':
    app.run_server(host='0.0.0.0',port=int(portID)) # Domino hosts all apps at 0.0.0.0:8888